<a href="https://colab.research.google.com/github/Binamra00/paint-paint/blob/main/notebooks/domain_inpaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paint-Paint: Domain Adaptive Image Inpainting
**Phase 3: Environment Setup & Bootstrapping**

This notebook is self-contained. It pulls the latest version of the `paint-paint` codebase from GitHub into the Colab environment, sets up the Python path for modular importing, and installs all strict dependencies.

In [2]:
import os
import sys

# 0. FORCE Colab to anchor to the default root directory
%cd /content

# 1. Clone the repository into Colab's local runtime
REPO_DIR = '/content/paint-paint'
if not os.path.exists(REPO_DIR):
    print("Cloning the codebase from GitHub...")
    !git clone https://github.com/Binamra00/paint-paint.git
else:
    print("Repository already exists. Pulling latest updates...")
    %cd "{REPO_DIR}"
    !git pull

# 2. Change working directory to the repo and add it to the Python path
%cd "{REPO_DIR}"
if REPO_DIR not in sys.path:
    sys.path.append(REPO_DIR)

# 3. Install strict dependencies
!pip install -q -r requirements.txt
print("Environment Ready!")

/content
Cloning the codebase from GitHub...
Cloning into 'paint-paint'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 54 (delta 19), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 22.68 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/paint-paint
Environment Ready!


## Pipeline Stage 1: Data Ingestion & Reproducibility
To ensure experiments can be globally reproduced, all random number generators are strictly locked. We then execute our custom `download_oxford.py` module to fetch the dataset.

In [3]:
import torch
import numpy as np
import random

# Lock seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

# Execute the custom data ingestion module
!python data/download_oxford.py

2026-02-28 19:20:17,893 - INFO - Downloading Oxford 102 Flowers from https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz...
2026-02-28 19:20:54,430 - INFO - Download complete!
2026-02-28 19:20:54,431 - INFO - Extracting /content/paint-paint/data/raw/102flowers.tgz to /content/paint-paint/data/raw/oxford_102...
/content/paint-paint/data/download_oxford.py:43: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=EXTRACT_DIR)
2026-02-28 19:20:59,865 - INFO - Extraction complete! Dataset is ready for the pipeline.
